# Week 4: Introduction to LangGraph

## 📚 Session Overview

**Duration:** 2 hours  
**Week:** 4  
**Instructor-Led Session**

---

## 🎯 Learning Objectives

By the end of this session, you will be able to:
1. Understand what LangGraph is and when to use it
2. Create nodes, edges, and state in graphs
3. Implement conditional routing logic
4. Build stateful, cyclical workflows
5. Visualize and debug graph execution
6. Create practical multi-step agents

---

## 📋 Prerequisites

- ✅ Completed Week 1, 2, and 3
- ✅ Strong understanding of LangChain chains
- ✅ Familiarity with LCEL
- ✅ Understanding of state management

---

## ⏱️ Estimated Time

- Setup & Introduction: 10 minutes
- Section 1 (Why LangGraph): 20 minutes
- Section 2 (Core Concepts): 30 minutes
- Section 3 (Building Graphs): 35 minutes
- Section 4 (Conditional Logic): 20 minutes
- Wrap-up & Q&A: 5 minutes

---

## 🔧 Setup

In [ ]:
# Import required libraries
import os
from dotenv import load_dotenv
from typing import TypedDict, Annotated, Sequence
import operator

# LangChain imports
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate

# LangGraph imports
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Load environment variables
load_dotenv()

# Initialize LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

print("✅ Setup complete!")

---

# Section 1: Why LangGraph? (20 minutes)

## The Problem with Chains

**LangChain Chains** (LCEL) are great for **linear workflows**:

```
Input → Step 1 → Step 2 → Step 3 → Output
```

But they have limitations:

### ❌ What Chains Can't Do Well:

1. **Cycles/Loops:** Can't easily go back to previous steps
2. **Complex Branching:** Limited conditional logic
3. **Stateful Logic:** Hard to maintain complex state across steps
4. **Dynamic Paths:** Can't decide flow based on intermediate results
5. **Human-in-the-Loop:** No natural place to pause for input

---

## What is LangGraph?

**LangGraph** is a library for building **stateful, multi-actor applications** with LLMs.

### ✅ What LangGraph Enables:

1. **Cycles:** Loops and iterative refinement
2. **Conditional Edges:** Dynamic routing based on state
3. **Persistence:** Save and resume execution
4. **Parallel Execution:** Run multiple nodes simultaneously
5. **Human-in-the-Loop:** Pause for human approval/input
6. **Complex State:** Track multiple pieces of information

---

## When to Use LangGraph vs Chains

### Use **Chains** when:
- Linear workflow (A → B → C)
- No loops or cycles needed
- Simple conditional logic
- Single pass through data

**Examples:**
- Translate text
- Summarize document
- Simple Q&A
- Data transformation pipeline

### Use **LangGraph** when:
- Need cycles/loops
- Complex decision trees
- Multiple agents/actors
- Iterative refinement
- Human approval needed
- Long-running workflows

**Examples:**
- Research agent (search → analyze → search more)
- Code reviewer (review → fix → review again)
- Customer support router (classify → route → escalate)
- Content moderation (check → human review → approve)

---

## Real-World Example

**Scenario:** Content Generation with Quality Control

**With Chains (Linear):**
```
Generate Draft → Polish → Done
```
❌ Problem: Can't iterate if quality is low

**With LangGraph (Cyclical):**
```
Generate Draft → Check Quality
                      ↓
           Good? → Yes → Done
              ↓
             No → Improve Draft (loop back)
```
✅ Solution: Iterates until quality threshold met

---

---

# Section 2: LangGraph Core Concepts (30 minutes)

LangGraph has three main building blocks:
1. **State** - Data that flows through the graph
2. **Nodes** - Functions that process state
3. **Edges** - Connections between nodes

---

## 2.1: State - The Shared Memory

**State** is a shared data structure that flows through the graph.

### Defining State with TypedDict

State is defined using Python's `TypedDict`:

```python
class MyState(TypedDict):
    messages: list[str]      # List of messages
    count: int               # A counter
    user_name: str          # User's name
```

### State Reducers

Sometimes you want to **combine** values instead of **replacing** them.

**Without Reducer (Default - Replace):**
```python
class State(TypedDict):
    messages: list  # Each node replaces the entire list
```

**With Reducer (Combine):**
```python
class State(TypedDict):
    messages: Annotated[list, operator.add]  # Each node adds to the list
```

Common reducers:
- `operator.add` - Concatenate lists/strings
- Custom function - Your own logic

---

In [ ]:
# Example: Define a simple state
class SimpleState(TypedDict):
    input: str
    output: str
    step_count: int

# Example: State with reducer
class MessagesState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    current_step: str

print("✅ State schemas defined")

## 2.2: Nodes - Processing Functions

**Nodes** are Python functions that:
1. Receive the current state
2. Process/transform it
3. Return updates to the state

### Node Function Signature

```python
def my_node(state: MyState) -> MyState:
    """Process the state and return updates."""
    # Read from state
    current_value = state["some_key"]
    
    # Do some processing
    new_value = process(current_value)
    
    # Return state updates (partial update, not full state)
    return {"some_key": new_value}
```

### Key Points:
- Nodes are **pure functions** (no side effects when possible)
- They receive the **full state**
- They return **partial updates** (only changed fields)
- LangGraph merges the updates into the state

---

In [ ]:
# Example nodes
def node_a(state: SimpleState) -> SimpleState:
    """First processing step."""
    print(f"Node A: Processing '{state['input']}'")
    return {
        "output": f"Processed by A: {state['input']}",
        "step_count": state.get("step_count", 0) + 1
    }

def node_b(state: SimpleState) -> SimpleState:
    """Second processing step."""
    print(f"Node B: Further processing '{state['output']}'")
    return {
        "output": f"Processed by B: {state['output']}",
        "step_count": state.get("step_count", 0) + 1
    }

print("✅ Node functions defined")

## 2.3: Edges - Connecting Nodes

**Edges** define how nodes are connected.

### Types of Edges:

#### 1. **Normal Edges** (Direct connections)
```python
graph.add_edge("node_a", "node_b")  # Always go from A to B
```

#### 2. **Conditional Edges** (Dynamic routing)
```python
def route_logic(state):
    if state["score"] > 0.8:
        return "approve"
    else:
        return "reject"

graph.add_conditional_edges(
    "check_quality",
    route_logic,
    {
        "approve": "finalize",
        "reject": "improve"
    }
)
```

#### 3. **Entry Point**
```python
graph.set_entry_point("start_node")  # Where execution begins
```

#### 4. **END**
```python
from langgraph.graph import END
graph.add_edge("final_node", END)  # Terminates execution
```

---

---

# Section 3: Building Your First Graph (35 minutes)

Let's build a complete graph step by step.

## 3.1: Simple Linear Graph

Start with a simple graph: A → B → C

In [ ]:
# Define state
class LinearState(TypedDict):
    text: str
    steps: Annotated[list[str], operator.add]

# Define nodes
def step_1(state: LinearState) -> LinearState:
    """First step: capitalize."""
    return {
        "text": state["text"].upper(),
        "steps": ["Step 1: Capitalized"]
    }

def step_2(state: LinearState) -> LinearState:
    """Second step: add prefix."""
    return {
        "text": f"PROCESSED: {state['text']}",
        "steps": ["Step 2: Added prefix"]
    }

def step_3(state: LinearState) -> LinearState:
    """Third step: add suffix."""
    return {
        "text": f"{state['text']} [DONE]",
        "steps": ["Step 3: Added suffix"]
    }

# Create the graph
workflow = StateGraph(LinearState)

# Add nodes
workflow.add_node("step_1", step_1)
workflow.add_node("step_2", step_2)
workflow.add_node("step_3", step_3)

# Add edges (linear flow)
workflow.set_entry_point("step_1")
workflow.add_edge("step_1", "step_2")
workflow.add_edge("step_2", "step_3")
workflow.add_edge("step_3", END)

# Compile the graph
app = workflow.compile()

print("✅ Linear graph created")

In [ ]:
# Execute the graph
initial_state = {
    "text": "hello world",
    "steps": []
}

result = app.invoke(initial_state)

print("📊 Execution Result:")
print(f"Final Text: {result['text']}")
print(f"\nSteps Executed:")
for step in result['steps']:
    print(f"  - {step}")

## 3.2: Graph with Conditional Routing

Let's add decision-making to the graph.

In [ ]:
# Define state for routing example
class RoutingState(TypedDict):
    message: str
    category: str
    response: str

# Classifier node
def classify_message(state: RoutingState) -> RoutingState:
    """Classify the incoming message."""
    message = state["message"].lower()
    
    if "urgent" in message or "asap" in message:
        category = "urgent"
    elif "question" in message or "how" in message or "what" in message:
        category = "question"
    else:
        category = "general"
    
    print(f"📋 Classified as: {category}")
    return {"category": category}

# Handler nodes
def handle_urgent(state: RoutingState) -> RoutingState:
    """Handle urgent messages."""
    print("🚨 Handling urgent message")
    return {"response": "This has been escalated to priority support."}

def handle_question(state: RoutingState) -> RoutingState:
    """Handle questions."""
    print("❓ Handling question")
    return {"response": "Let me find the answer for you."}

def handle_general(state: RoutingState) -> RoutingState:
    """Handle general messages."""
    print("💬 Handling general message")
    return {"response": "Thank you for your message."}

# Routing function
def route_by_category(state: RoutingState) -> str:
    """Decide which node to route to based on category."""
    return state["category"]

# Create graph
routing_workflow = StateGraph(RoutingState)

# Add nodes
routing_workflow.add_node("classify", classify_message)
routing_workflow.add_node("urgent", handle_urgent)
routing_workflow.add_node("question", handle_question)
routing_workflow.add_node("general", handle_general)

# Set entry point
routing_workflow.set_entry_point("classify")

# Add conditional edges
routing_workflow.add_conditional_edges(
    "classify",
    route_by_category,
    {
        "urgent": "urgent",
        "question": "question",
        "general": "general"
    }
)

# All handlers go to END
routing_workflow.add_edge("urgent", END)
routing_workflow.add_edge("question", END)
routing_workflow.add_edge("general", END)

# Compile
routing_app = routing_workflow.compile()

print("✅ Routing graph created")

In [ ]:
# Test the routing graph
test_messages = [
    "URGENT: Server is down!",
    "What is your return policy?",
    "Thank you for the great service"
]

for msg in test_messages:
    print(f"\n{'='*60}")
    print(f"📨 Message: {msg}")
    result = routing_app.invoke({"message": msg, "category": "", "response": ""})
    print(f"✅ Response: {result['response']}")

## 3.3: Graph with Loops (Iterative Refinement)

Build a graph that loops until a condition is met.

In [ ]:
# Define state for iterative task
class IterativeState(TypedDict):
    draft: str
    quality_score: float
    iteration: int
    max_iterations: int

# Generate draft
def generate_draft(state: IterativeState) -> IterativeState:
    """Generate or improve the draft."""
    iteration = state.get("iteration", 0) + 1
    
    if iteration == 1:
        draft = "This is a basic draft."
        print(f"📝 Generated initial draft (iteration {iteration})")
    else:
        draft = state["draft"] + " Enhanced with more detail."
        print(f"✏️ Improved draft (iteration {iteration})")
    
    return {
        "draft": draft,
        "iteration": iteration
    }

# Evaluate quality
def evaluate_quality(state: IterativeState) -> IterativeState:
    """Evaluate the quality of the draft."""
    # Simple quality scoring based on length
    score = min(len(state["draft"]) / 100, 1.0)
    print(f"📊 Quality score: {score:.2f}")
    return {"quality_score": score}

# Routing logic
def should_continue(state: IterativeState) -> str:
    """Decide whether to continue improving or finish."""
    if state["quality_score"] >= 0.7:
        print("✅ Quality threshold met!")
        return "finish"
    elif state["iteration"] >= state["max_iterations"]:
        print("⚠️ Max iterations reached")
        return "finish"
    else:
        print("🔄 Continuing to improve...")
        return "continue"

# Create iterative graph
iterative_workflow = StateGraph(IterativeState)

# Add nodes
iterative_workflow.add_node("generate", generate_draft)
iterative_workflow.add_node("evaluate", evaluate_quality)

# Set entry point
iterative_workflow.set_entry_point("generate")

# Add edges
iterative_workflow.add_edge("generate", "evaluate")

# Add conditional edge with loop
iterative_workflow.add_conditional_edges(
    "evaluate",
    should_continue,
    {
        "continue": "generate",  # Loop back!
        "finish": END
    }
)

# Compile
iterative_app = iterative_workflow.compile()

print("✅ Iterative graph created")

In [ ]:
# Execute iterative graph
initial_state = {
    "draft": "",
    "quality_score": 0.0,
    "iteration": 0,
    "max_iterations": 5
}

print("🚀 Starting iterative improvement...\n")
result = iterative_app.invoke(initial_state)

print("\n" + "="*60)
print("📊 Final Result:")
print(f"Iterations: {result['iteration']}")
print(f"Quality Score: {result['quality_score']:.2f}")
print(f"Final Draft: {result['draft']}")

---

# Section 4: Advanced Graph Features (20 minutes)

## 4.1: Graph Visualization

LangGraph can generate visual representations of your graphs.

In [ ]:
# Get Mermaid diagram
try:
    from IPython.display import Image, display
    
    # Generate graph visualization
    display(Image(routing_app.get_graph().draw_mermaid_png()))
except Exception as e:
    # If visualization fails, show text representation
    print("Graph structure (Mermaid):")
    print(routing_app.get_graph().draw_mermaid())

## 4.2: Streaming Graph Execution

Stream updates as the graph executes.

In [ ]:
# Stream execution
print("🔄 Streaming graph execution:\n")

initial_state = {"message": "How do I reset my password?", "category": "", "response": ""}

for event in routing_app.stream(initial_state):
    for node_name, node_output in event.items():
        print(f"📍 Node '{node_name}':")
        print(f"   Output: {node_output}")
        print()

## 4.3: Checkpointing (State Persistence)

Save graph state and resume later.

In [ ]:
# Create graph with checkpointing
memory = MemorySaver()

# Recompile with checkpointer
checkpointed_app = iterative_workflow.compile(checkpointer=memory)

# Configuration for thread
config = {"configurable": {"thread_id": "1"}}

# Run with checkpointing
initial_state = {
    "draft": "",
    "quality_score": 0.0,
    "iteration": 0,
    "max_iterations": 3
}

result = checkpointed_app.invoke(initial_state, config=config)

print("✅ Graph executed with checkpointing")
print(f"Final iteration: {result['iteration']}")

# Get checkpoint history
print("\n📜 Checkpoint History:")
for state in checkpointed_app.get_state_history(config):
    print(f"  Iteration {state.values.get('iteration', 0)}: Score {state.values.get('quality_score', 0):.2f}")

## 4.4: Practical Example - Customer Support Router

Complete example with LLM integration.

In [ ]:
# State for customer support
class SupportState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    query: str
    intent: str
    response: str

# Classify intent with LLM
def classify_intent(state: SupportState) -> SupportState:
    """Use LLM to classify user intent."""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Classify the user query into one of: technical, billing, general. Respond with only one word."),
        ("human", "{query}")
    ])
    
    chain = prompt | llm
    result = chain.invoke({"query": state["query"]})
    intent = result.content.strip().lower()
    
    print(f"🤖 Classified intent: {intent}")
    
    return {
        "intent": intent,
        "messages": [AIMessage(content=f"Classified as {intent}")]
    }

# Technical support
def technical_support(state: SupportState) -> SupportState:
    """Handle technical queries."""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a technical support specialist. Provide clear technical guidance."),
        ("human", "{query}")
    ])
    
    chain = prompt | llm
    result = chain.invoke({"query": state["query"]})
    
    return {
        "response": result.content,
        "messages": [AIMessage(content=result.content)]
    }

# Billing support
def billing_support(state: SupportState) -> SupportState:
    """Handle billing queries."""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a billing specialist. Help with payment and subscription issues."),
        ("human", "{query}")
    ])
    
    chain = prompt | llm
    result = chain.invoke({"query": state["query"]})
    
    return {
        "response": result.content,
        "messages": [AIMessage(content=result.content)]
    }

# General support
def general_support(state: SupportState) -> SupportState:
    """Handle general queries."""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful customer service agent. Provide friendly assistance."),
        ("human", "{query}")
    ])
    
    chain = prompt | llm
    result = chain.invoke({"query": state["query"]})
    
    return {
        "response": result.content,
        "messages": [AIMessage(content=result.content)]
    }

# Route based on intent
def route_support(state: SupportState) -> str:
    """Route to appropriate support node."""
    intent = state["intent"]
    if "technical" in intent:
        return "technical"
    elif "billing" in intent:
        return "billing"
    else:
        return "general"

# Build support graph
support_workflow = StateGraph(SupportState)

# Add nodes
support_workflow.add_node("classify", classify_intent)
support_workflow.add_node("technical", technical_support)
support_workflow.add_node("billing", billing_support)
support_workflow.add_node("general", general_support)

# Set entry
support_workflow.set_entry_point("classify")

# Add conditional routing
support_workflow.add_conditional_edges(
    "classify",
    route_support,
    {
        "technical": "technical",
        "billing": "billing",
        "general": "general"
    }
)

# All end
support_workflow.add_edge("technical", END)
support_workflow.add_edge("billing", END)
support_workflow.add_edge("general", END)

# Compile
support_app = support_workflow.compile()

print("✅ Customer support graph created")

In [ ]:
# Test customer support router
test_queries = [
    "My app keeps crashing when I try to upload files",
    "I was charged twice for my subscription",
    "What are your business hours?"
]

for query in test_queries:
    print(f"\n{'='*70}")
    print(f"❓ Query: {query}")
    print()
    
    result = support_app.invoke({
        "query": query,
        "messages": [],
        "intent": "",
        "response": ""
    })
    
    print(f"💡 Response:\n{result['response']}")

---

# 🎯 Summary & Key Takeaways

## What We Learned:

### 1. **Why LangGraph**
- Chains are linear, LangGraph handles complexity
- Enables cycles, conditionals, and stateful workflows
- Perfect for multi-step agents and iterative tasks

### 2. **Core Concepts**
- **State:** TypedDict with optional reducers
- **Nodes:** Functions that process and update state
- **Edges:** Connect nodes (normal and conditional)

### 3. **Building Graphs**
- StateGraph creation and compilation
- Adding nodes and edges
- Setting entry points and endpoints
- Conditional routing logic

### 4. **Advanced Features**
- Graph visualization
- Streaming execution
- Checkpointing and persistence
- LLM integration in nodes

---

## 📝 Next Steps:

### Exercises for This Week:

**Exercise 1 (Due Monday):** `02_exercise_content_moderation.ipynb`
- Build content moderation pipeline
- Implement multi-check system
- Add conditional routing

**Exercise 2 (Due Friday):** `03_exercise_research_agent.ipynb`
- Create research agent with loops
- Implement iterative search
- Add quality checks and guardrails

---

## 🤔 Reflection Questions:

1. When should you use LangGraph vs LangChain chains?
2. How do state reducers work and when are they useful?
3. What are the benefits of conditional edges?
4. How can checkpointing improve user experience?

---

## 📚 Additional Resources:

- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [LangGraph Tutorials](https://github.com/langchain-ai/langgraph/tree/main/examples)
- [StateGraph API Reference](https://langchain-ai.github.io/langgraph/reference/graphs/)

---

**Next Week:** Advanced LangGraph with Human-in-the-Loop! 🚀